In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = (
    'cuda'
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'
)

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        return x

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
X = torch.rand(1, 28, 28, device=device)
output = model(X)
print(output)
pred_prob = nn.Softmax(dim=1)(output)
y_pred = pred_prob.argmax(1)
print(y_pred)

tensor([[ 0.0597, -0.1197,  0.0369,  0.0078,  0.0697,  0.0181,  0.0200,  0.0008,
         -0.0286, -0.0178]], device='mps:0', grad_fn=<LinearBackward0>)
tensor([4], device='mps:0')


In [7]:
# Flatten
input_image = torch.rand(3, 28, 28)
print(input_image.size())
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 28, 28])
torch.Size([3, 784])


In [8]:
# Linear
layer1 = nn.Linear(in_features=28 * 28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
# Non-linear activations are what create the complex mappings between the model's inputs and outputs.
# They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.
print(f'Before ReLU: {hidden1}\n\n')
hidden1 = nn.ReLU()(hidden1)
print(f'After ReLU: {hidden1}')

Before ReLU: tensor([[ 0.5304, -0.0387,  0.1457, -0.0368, -0.1786,  0.1699, -0.2958,  0.1117,
         -0.1328, -0.5773, -0.1258, -0.0054, -0.0102,  0.0899,  0.4953, -0.1897,
         -0.0718, -0.2188,  0.0777, -0.1353],
        [ 0.4682,  0.3442, -0.0826,  0.1612, -0.6828,  0.2092, -0.0319, -0.0601,
         -0.5380, -0.4898,  0.1628, -0.3051, -0.2856,  0.0749,  0.6120, -0.0749,
         -0.3890, -0.2893, -0.0503,  0.1147],
        [ 0.3044,  0.1364,  0.1781, -0.6261, -0.2257,  0.2489, -0.2052, -0.2766,
         -0.6885, -0.2813,  0.0883, -0.2700, -0.0558,  0.0152,  0.6006, -0.2427,
         -0.3006, -0.4396, -0.0691,  0.0249]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.5304, 0.0000, 0.1457, 0.0000, 0.0000, 0.1699, 0.0000, 0.1117, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0899, 0.4953, 0.0000, 0.0000, 0.0000,
         0.0777, 0.0000],
        [0.4682, 0.3442, 0.0000, 0.1612, 0.0000, 0.2092, 0.0000, 0.0000, 0.0000,
         0.0000, 0.1628, 0.0000, 0.0000, 0.0749, 0.61

In [10]:
# Model parameters
print(f'Model structure: {model}\n\n')
for name, param in model.named_parameters():
    print(f'Lyaer: {name} | Size: {param.size()} | Values: {param[:2]}\n')

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Lyaer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0229, -0.0332,  0.0101,  ...,  0.0224,  0.0123, -0.0100],
        [ 0.0011, -0.0010,  0.0355,  ..., -0.0063, -0.0218,  0.0257]],
       device='mps:0', grad_fn=<SliceBackward0>)

Lyaer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([0.0318, 0.0139], device='mps:0', grad_fn=<SliceBackward0>)

Lyaer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0291,  0.0176, -0.0081,  ...,  0.0436,  0.0043, -0.0220],
        [-0.0276,  0.0052, -0.0415,  ...,  0.0093,  0.0008,  0.0255]],
       device='mps:0', grad_fn=<SliceBackwar